# Conducting Automated/Autonomous Experiments with MADSci

This aims to teach you how to define and run automated/autonomous experiment applications using a MADSci Workcell.

## Goals

After completing this notebook, you should understand how to define and use a MADSci `ExperimentApplication` to manage data, resources, logging and workflows for you scientific applications. 


## What is an Experiment Application?

The MADSci software architecture uses a number of modular components, called managers, to organize the workflows, resources and datapoints that we will use to actually do science. In order to interface with the manager, we provide clients with predefined functions that make the manager functionality easy to understand and use. The `ExperimentApplication` class provided by MADSci leverages the Lab manager to retrieve configuration information and create clients for all managers used in a MADSci experiment. It also provides some utility functions that make it easier to run experiments and handle data locally. Finally, `ExperimentApplication`s are able to operate as Nodes, allowing you to incorporate custom functionality into workflows, including data processing and AI control

In [ ]:
# Install dependencies
%pip install madsci.experiment_application

This demo will assume that you have the Example Lab found in the MADSci repo installed and running on your local machine. If this is not the case, follow the instructions found in the top level MADSci readme, including running Just Up


## Setup

Below is an example class inheriting from the Experiment Application class. 

In [ ]:
from madsci.common.types.experiment_types import ExperimentDesign
from madsci.experiment_application.experiment_application import ExperimentApplication


class ExampleExperimentApplication(ExperimentApplication):
    """An example experiment application."""

    experiment_design = ExperimentDesign(
        experiment_name="Example Experiment",
        experiment_description="An Example Experiment",
    )


experiment_application = ExampleExperimentApplication(
    lab_server_url="http://localhost:8000"
)
print(experiment_application.experiment_design)  # noqa

An Experiment Application contains an `ExperimentDesign` object. This can be as simple as the name and description of the experiment, but can also contain checks to ensure that the resources of the lab are in the correct state to begin a run of the experiment, though this functionality is still being tested and refined


### What can we get from an Experiment application?

Clients! The experiment application gets the lab configuration from the Lab server, and uses this to creat clients for all of the MADSci managers available. Clients are python objects that provide structured ways to utilize the MADSi manager functionality. Below are examples of some of the most important clients and some basics of their usage






#### Workcell Client

The Workcell Client interacts with the workcell manager, and can be used to send and monitor workflows, add nodes to the workcell and request state information about the workcell and its nodes. Below are some examples of this functionality


In [ ]:
from pathlib import Path

from madsci.common.types.step_types import StepDefinition
from madsci.common.types.workflow_types import WorkflowDefinition

##An Example Workflow, defined in code. This workflow can also be read from a yaml file
workflow_definition = WorkflowDefinition(
    name="Example Workflow 2",
    steps=[
        StepDefinition(
            name="Run Liquidhandler Protocol",
            description="Run the Liquidhandler",
            node="liquidhandler_1",
            action="run_protocol",
            files={"protocol": str(Path("../protocols/protocol.py").resolve())},
        ),
        StepDefinition(
            name="Transfer from liquid handler to plate reader",
            description="Transfer an asset from the liquid handler to the plate reader",
            node="robotarm_1",
            action="transfer",
            locations={
                "source": "liquidhandler_1.deck_1",
                "target": "platereader_1.plate_carriage",
            },
        ),
        StepDefinition(
            name="run platereader measurement",
            description="measure a well on the plate reader",
            node="platereader_1",
            action="read_well",
        ),
        StepDefinition(
            name="Transfer from  plate reader to liquid handler",
            description="Transfer an asset from the liquid handler to the plate reader",
            node="robotarm_1",
            action="transfer",
            locations={
                "source": "platereader_1.plate_carriage",
                "target": "liquidhandler_1.deck_1",
            },
        ),
    ],
)

# Run a workflow synchronously, this will wait for the workflow to complete on this line
experiment_application.workcell_client.submit_workflow(workflow_definition)

# Run a workflow asynchronously, and then query its status until it is complete
workflow_run = experiment_application.workcell_client.submit_workflow(
    workflow_definition, await_completion=False
)
while not workflow_run.status.terminal:
    print(f"Workflow status: {workflow_run.status}")  # noqa
    workflow_run = experiment_application.workcell_client.query_workflow(
        workflow_run.workflow_id
    )